# 1.Load data

In [1]:
import keras
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.python.keras.backend import get_session

In [2]:
import pandas as pd
import imblearn
import numpy as np

In [3]:
data = pd.read_csv('./OUTPUT/dataset.csv', encoding= 'unicode_escape')
data

,Sentence,Word,Tag
0,Sentence: 1,RECORD,0
1,Sentence: 2,OC,0
2,NaN,AM,0
3,NaN,gallstone,0
4,NaN,pancreatitis,0
...,...,...,...
949802,NaN,M.D.,0
949803,Sentence: 132094,END,0
949804,NaN,OF,0
949805,NaN,DISCHARGE,0


## Groupby

In [4]:
data_fillna = data.fillna(method='ffill', axis=0)
data_group = data_fillna.groupby(['Sentence'],as_index=False
                                )['Word', 'Tag'].agg(lambda x: list(x))

#data_fillna
data_group

<ipython-input-4-73aff18756c8>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_group = data_fillna.groupby(['Sentence'],as_index=False


,Sentence,Word,Tag
0,Sentence: 1,[RECORD],[0]
1,Sentence: 10,"[WILL, D/C, ORDER, BE, USED, AS, THE, D/C, SUM...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,Sentence: 100,"[prandial, N/V/severe, upper, abdominal, pain....","[0, 1, 0, 1, 1, 0, 0, 0, 0]"
3,Sentence: 1000,"[normal, limits., Cardiac, catheterization, da...","[0, 0, 0, 0, 0, 0, 0, 0]"
4,Sentence: 10000,"[year, old, Black, female, with, significant, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...
132089,Sentence: 99995,"[Height, foot, inch, and, weight, kg., Tempera...","[0, 0, 0, 0, 0, 0, 0]"
132090,Sentence: 99996,"[degrees, heart, rate, and, sinus, blood, pres...","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
132091,Sentence: 99997,"[blood, pressure, left, arm, and, oxygen, satu...","[0, 0, 0, 0, 0, 0, 0]"
132092,Sentence: 99998,"[No, carotid, bruits, regular, rate, and, rhyt...","[0, 0, 0, 0, 0, 0, 0, 0, 0]"


## Padding

In [5]:
texts = data_group['Word'].tolist()  
labels = data_group['Tag'].tolist()  

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

pad_tokens = pad_sequences(sequences, maxlen=49, dtype='int32', padding='post', value= 0)
print(pad_tokens)
pad_tags = pad_sequences(labels, maxlen=49, dtype='int32', padding='post', value= 0)
train_tokens, test_tokens, train_tags, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.3, train_size=0.7, random_state=2020)
print(pad_tags)
print('Shape of data tensor:', pad_tokens.shape)
print('Shape of label tensor:', pad_tags.shape)

Found 34275 unique tokens.
[[  115     0     0 ...     0     0     0]
 [   44   145   106 ...     0     0     0]
 [ 6315 15212   259 ...     0     0     0]
 ...
 [   42    70    33 ...     0     0     0]
 [   13   421  1398 ...     0     0     0]
 [  327   561  1373 ...     0     0     0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Shape of data tensor: (132094, 49)
Shape of label tensor: (132094, 49)


## GLOVE Embedding Layer

In [7]:
import os

embeddings_index = {}
f = open(os.path.join(r'./GLOVE', 'glove.6B.300d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [8]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [9]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            300,
                            weights=[embedding_matrix],
                            input_length=49,
                            trainable=True)

## Model

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model
from keras import backend as K
import keras as keras
from keras.datasets import mnist
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from keras import metrics
from sklearn.utils import class_weight

In [11]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [12]:
def get_bilstm_lstm_model():
    model = Sequential()

    # Add Embedding layer
    model.add(embedding_layer)
    # Add bidirectional LSTM
    model.add(Bidirectional(LSTM(units=32, return_sequences=True, dropout=0.5, recurrent_dropout=0.5), merge_mode = 'concat'))

    
    # Add LSTM
    model.add(LSTM(units=32, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))
    model.add(LSTM(units=32, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))
    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(1, activation="sigmoid")))
    
    #Optimiser 
    adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
    
    # Compile model
    model.compile(loss='binary_crossentropy', sample_weight_mode="temporal", optimizer='adam', metrics=['acc', precision_m, recall_m, f1_m])
    model.summary()
    
    return model

In [13]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 999)

fold_number = 1
f1_per_fold = []
recall_per_fold = []
precision_per_fold = []
acc_per_fold = []
loss_per_fold = []

for i, (train_index, val_index) in enumerate(skf.split(pad_tokens, np.sum(pad_tags, axis = 1))):
    
    print("Training on fold " + str(i+1) + "/10..........")
    
    #Split training set and validation set
    x_train, x_val = pad_tokens[train_index], pad_tokens[val_index]
    y_train, y_val = pad_tags[train_index], pad_tags[val_index]
    
    #Assigning sample weights in training set
    print(str(fold_number) + ": started assigning sample weights")
    weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(np.ravel(y_train,order='C')),
                                                 np.ravel(y_train,order='C'))
    
    train_tags2 = np.copy(y_train)
    train_tokens2 = np.copy(x_train)
    train_tags2 = train_tags2.astype(float)
    
    indexTotal = 0
    for tags in train_tags2:
        indexTags = 0
        for symptom in tags:
            if symptom == 1:
                train_tags2[indexTotal][indexTags] = float(weights[1]+6.00)
            else:
                train_tags2[indexTotal][indexTags] = float(weights[0])
            indexTags = indexTags+1
        indexTotal = indexTotal + 1
   
    weights = train_tags2.reshape((-1, 49, 1))
    print(str(fold_number) + ": finished assigning sample weights")
    
    #Getting Model Architecture
    model = get_bilstm_lstm_model()
    
    #Running Model
    history = model.fit(x_train, y_train, sample_weight = weights, batch_size=64, verbose=1, epochs=20)
    
    #Evaluate model
    scores = model.evaluate(x_val, y_val, verbose = 0)
    print(f'Score for fold {fold_number}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}; {model.metrics_names[2]} of {scores[2]}; {model.metrics_names[3]} of {scores[3]}; {model.metrics_names[4]} of {scores[4]} %')
    f1_per_fold.append(scores[4])
    recall_per_fold.append(scores[3])
    precision_per_fold.append(scores[2])
    acc_per_fold.append(scores[1])
    loss_per_fold.append(scores[0])
    
    #Increase fold number
    fold_number = fold_number + 1

C:\Users\jonat\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


Training on fold 1/10..........
1: started assigning sample weights


C:\Users\jonat\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=[0 0 0 ... 0 0 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


1: finished assigning sample weights
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 49, 300)           10282800  
                                                                 
 bidirectional (Bidirectiona  (None, 49, 64)           85248     
 l)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 49, 32)            12416     
                                                                 
 lstm_2 (LSTM)               (None, 49, 32)            8320      
                                                                 
 time_distributed (TimeDistr  (None, 49, 1)            33        
 ibuted)                                                         
                                                                 
Total params: 10,38

1858/1858 [==============================] - 573s 308ms/step - loss: 0.0017 - acc: 0.9995 - precision_m: 0.9252 - recall_m: 0.9151 - f1_m: 0.9146
Epoch 9/20
1858/1858 [==============================] - 574s 309ms/step - loss: 0.0017 - acc: 0.9995 - precision_m: 0.9260 - recall_m: 0.9197 - f1_m: 0.9179
Epoch 10/20
1858/1858 [==============================] - 574s 309ms/step - loss: 0.0015 - acc: 0.9995 - precision_m: 0.9330 - recall_m: 0.9269 - f1_m: 0.9254
Epoch 11/20
1858/1858 [==============================] - 573s 308ms/step - loss: 0.0015 - acc: 0.9995 - precision_m: 0.9293 - recall_m: 0.9200 - f1_m: 0.9189
Epoch 12/20
1858/1858 [==============================] - 573s 308ms/step - loss: 0.0015 - acc: 0.9995 - precision_m: 0.9304 - recall_m: 0.9266 - f1_m: 0.9232
Epoch 13/20
1858/1858 [==============================] - 571s 308ms/step - loss: 0.0014 - acc: 0.9995 - precision_m: 0.9286 - recall_m: 0.9202 - f1_m: 0.9196
Epoch 14/20
1858/1858 [==============================] - 577s 310

1858/1858 [==============================] - 699s 370ms/step - loss: 0.0120 - acc: 0.9973 - precision_m: 0.6782 - recall_m: 0.5508 - f1_m: 0.5811
Epoch 2/20
1858/1858 [==============================] - 692s 372ms/step - loss: 0.0034 - acc: 0.9990 - precision_m: 0.8608 - recall_m: 0.8534 - f1_m: 0.8466
Epoch 3/20
1858/1858 [==============================] - 693s 373ms/step - loss: 0.0025 - acc: 0.9993 - precision_m: 0.8958 - recall_m: 0.8867 - f1_m: 0.8840
Epoch 4/20
1858/1858 [==============================] - 692s 372ms/step - loss: 0.0020 - acc: 0.9994 - precision_m: 0.9095 - recall_m: 0.9022 - f1_m: 0.8992
Epoch 5/20
1858/1858 [==============================] - 692s 372ms/step - loss: 0.0018 - acc: 0.9994 - precision_m: 0.9174 - recall_m: 0.9081 - f1_m: 0.9061
Epoch 6/20
1858/1858 [==============================] - 692s 372ms/step - loss: 0.0016 - acc: 0.9995 - precision_m: 0.9269 - recall_m: 0.9186 - f1_m: 0.9173
Epoch 7/20
1858/1858 [==============================] - 692s 372ms/st

C:\Users\jonat\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=[0 1 0 ... 0 0 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


8: finished assigning sample weights
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 49, 300)           10282800  
                                                                 
 bidirectional_7 (Bidirectio  (None, 49, 64)           85248     
 nal)                                                            
                                                                 
 lstm_22 (LSTM)              (None, 49, 32)            12416     
                                                                 
 lstm_23 (LSTM)              (None, 49, 32)            8320      
                                                                 
 time_distributed_7 (TimeDis  (None, 49, 1)            33        
 tributed)                                                       
                                                                 
Total params: 10,

In [14]:
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print("-----------")
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]} - Precision: {precision_per_fold[i]} - Recall: {recall_per_fold[i]} - F1: {f1_per_fold[i]}%')
print('------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Precision: {np.mean(precision_per_fold)} (+- {np.std(precision_per_fold)})')
print(f'> Recall: {np.mean(recall_per_fold)} (+- {np.std(recall_per_fold)})')
print(f'> F1: {np.mean(f1_per_fold)} (+- {np.std(f1_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------')

Score per fold
-----------
> Fold 1 - Loss: 0.008731890469789505 - Accuracy: 0.9979530572891235 - Precision: 0.6661249399185181 - Recall: 0.6925951838493347 - F1: 0.6524550914764404%
-----------
> Fold 2 - Loss: 0.004619639832526445 - Accuracy: 0.9988755583763123 - Precision: 0.7796337008476257 - Recall: 0.7546238303184509 - F1: 0.7495155334472656%
-----------
> Fold 3 - Loss: 0.0038018280174583197 - Accuracy: 0.9990332126617432 - Precision: 0.7725365161895752 - Recall: 0.7495949268341064 - F1: 0.7432667016983032%
-----------
> Fold 4 - Loss: 0.0030736096668988466 - Accuracy: 0.9992074966430664 - Precision: 0.8228378295898438 - Recall: 0.7872909903526306 - F1: 0.7916290760040283%
-----------
> Fold 5 - Loss: 0.0025577140040695667 - Accuracy: 0.9992725253105164 - Precision: 0.8347023725509644 - Recall: 0.8311446905136108 - F1: 0.8211787939071655%
-----------
> Fold 6 - Loss: 0.002410390181466937 - Accuracy: 0.9993482232093811 - Precision: 0.8420485258102417 - Recall: 0.8117166757583618 

## Self Check

In [74]:
predict_tags = model.predict(test_tokens)

In [75]:
truePositive = 0 
falsePositive = 0
trueNegative = 0 
falseNegative = 0
index = 0
for tag, predTag in zip(test_tags, predict_tags):
    for symptomTag, symptomPred in zip (tag, predTag):
        if symptomPred >= 0.50 and symptomTag == 1:
            truePositive = truePositive + 1
        elif symptomPred >= 0.50 and symptomTag == 0:
            falsePositive = falsePositive + 1
        elif symptomPred < 0.50 and symptomTag == 0:
            trueNegative = trueNegative + 1
        elif symptomPred < 0.50 and symptomTag == 1:
            falseNegative = falseNegative + 1

In [76]:
print('True postitive: ' + str(truePositive))
print('False postitive: ' + str(falsePositive))
print('True negative: ' + str(trueNegative))
print('False negative: ' + str(falseNegative))

True postitive: 6887
False postitive: 359
True negative: 1934322
False negative: 253


In [77]:
precision = (truePositive)/(truePositive+falsePositive)
recall = (truePositive)/(truePositive+falseNegative)

In [78]:
print('Accuracy: ' + str((truePositive+trueNegative)/(truePositive+trueNegative+falsePositive+falseNegative)))
print('Precision: ' + str((truePositive)/(truePositive+falsePositive)))
print('Recall: ' + str((truePositive)/(truePositive+falseNegative)))
print('F1: ' + str((2*precision*recall)/(precision+recall)))

Accuracy: 0.999684831918081
Precision: 0.9504554236820315
Recall: 0.9645658263305322
F1: 0.9574586403447796


In [79]:
model.evaluate(pad_tokens, pad_tags)

4128/4128 [==============================] - 215s 52ms/step - loss: 0.0010 - acc: 0.9997 - precision_m: 0.6882 - recall_m: 0.6997 - f1_m: 0.6890


[0.0010336712002754211,
 0.9996744990348816,
 0.6881966590881348,
 0.6996971368789673,
 0.6890236735343933]